In [19]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy.ext.declarative import declared_attr
from sqlalchemy import Column, Integer, String, Enum, Date, Sequence, DateTime, Boolean
from sqlalchemy.orm.exc import NoResultFound

engine = create_engine('sqlite:///:memory:', echo=True)

In [2]:
Base = declarative_base()

In [18]:
import datetime

class TimestampMixin(object):
    created_at = Column(DateTime, default=datetime.datetime.utcnow)
    
class TableNameIdMixin(object):
    @declared_attr
    def __tablename__(cls):
        return cls.__name__.lower()+'s'
    
    @declared_attr
    def id(cls):
        return Column(Integer, Sequence('{}_id_seq'.format(cls.__tablename__)), primary_key=True)

class PersonMixin(object):
    firstname = Column(String(50))
    lastname = Column(String(50))

In [ ]:
class User(Base, TableNameIdMixin, PersonMixin):
    username = Column(String(20), nullable=False, unique=True)
    password_hash = Column(String(200), nullable=False)
    
    def __repr__(self):
        return "<User(firstname={}, lastname={}, id={})>".format(
            self.firstname, self.lastname, self.id)
    
user_group_association = None

class Group(Base, TableNameIdMixin):
    name = Column(String(20), nullable=False)
    
    
class Dataset(Base, TableNameIdMixin):
    info = Column(String(10000), nullable=False)
    name = Column(String(50))
    
class DatasetRights(Base, TableNameIdMixin):
    group_id = Column(Integer, ForeignKey('groups.id'), nullable=False)
    dataset_id = Column(Integer, ForeignKey('datasets.id'), nullable=False)
    read = Column(Boolean, nullable=False, default=False)
    write = Column(Boolean, nullable=False, default=False)
    delete = Column(Boolean, nullable=False, default=False)
    

class Patient(Base, TableNameIdMixin, PersonMixin):
    gender = Column(Enum('M', 'F'))
    birthday = Column(Date, nullable=False)
    
    def __repr__(self):
        return "<Patient(firstname={}, lastname={}, id={}, gender={}, birthday={})>".format(
            self.firstname, self.lastname, self.id, self.gender, self.birthday)
    
class Visit(Base, TableNameIdMixin):
    patient_id = Column(Integer, ForeignKey('patients.id'), nullable=False)
    visit_date = Column(Date, nullable=False)
    
class Image(Base, TableNameIdMixin):
    patient_id = Column(Integer, ForeignKey('patients.id'), nullable=False)
    visit_id = Column(Integer, ForeignKey('visits.id'), nullable=False)
    filepath = Column(String(500),  nullable=False)
    
class ImageMeta(Base, TableNameIdMixin):
    modality = Column(Enum('CFP', 'OCT', 'NIR'),  nullable=False)
    eye = Column(Enum('Left', 'Right'),  nullable=False)
    image_id = None
    
class Registration(Base, TableNameIdMixin):
    image1_id = Column(Integer, ForeignKey('images.id'), nullable=False)
    image2_id = Column(Integer, ForeignKey('images.id'), nullable=False)
    registration_type = None
    parameters = None
    
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)    
    user = relationship("User", foreign_keys=[user_id])
    
class Correspondence(Base, TableNameIdMixin, TimestampMixin):
    registration_id = Column(Integer, ForeignKey('registrations.id'), nullable=False)
    point1_id = Column(Integer, ForeignKey('points.id'), nullable=False)
    point2_id = Column(Integer, ForeignKey('points.id'), nullable=False)
    
    
    registration = relationship("Registration", foreign_keys=[registration_id])
    point1 = relationship("Point", foreign_keys=[point1_id])
    point2 = relationship("Point", foreign_keys=[point2_id])
    

class Point(Base, TableNameIdMixin)
    xvalue = Column(Integer, nullable=False)
    yvalue = Column(Integer, nullable=False)
    
    

class LineSegmentation(Base, TableNameIdMixin):
    image_id = None
    
    # RPE, HRF, Fovea ...
    region = None
    
    # Guideline 
    guideline = None    
    
class AreaSegmentation(Base, TableNameIdMixin):
    pass
    
class Shapesegmentation(Base, TableNameIdMixin):
    pass
    
    
    



    

In [29]:
import hashlib, binascii, os
def hash_password(password):
    """Hash a password for storing."""
    salt = hashlib.sha256(os.urandom(60)).hexdigest().encode('ascii')
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'),
                                  salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')

def verify_password(stored_password, provided_password):
    """Verify a stored password against one provided by user"""
    salt = stored_password[:64]
    stored_password = stored_password[64:]
    pwdhash = hashlib.pbkdf2_hmac('sha512',
                                  provided_password.encode('utf-8'),
                                  salt.encode('ascii'),
                                  100000)
    pwdhash = binascii.hexlify(pwdhash).decode('ascii')
    return pwdhash == stored_password



In [35]:
hashed_pw = hash_password("testbla")

verify_password(hashed_pw, "tesstbla")
len(hashed_pw)

192

In [ ]:
Base.metadata.create_all(engine)